<a href="https://colab.research.google.com/github/jsroa15/BCG/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Feature Engineering**

In this section we are going to explore how to create the best features for the churn prediction in PowerCo.

The steps we are going to folow are:
1.   Create new features (domain knowlodge, grouping, interactions, extracting dates, drop for correlations)       
2.   Split data into Train and Test
3.   Outlier detection
4.   Possible transformations
8.   Scaling features

# **Import modules and load data**

In [412]:
#Import modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

In [413]:
#Load data

df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BCG/df_customer.csv')
hist=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BCG/df_hist.csv')

In [414]:
to_convert=['date_activ','date_end','date_modif_prod','date_renewal']

for i in to_convert:
  df[i]=pd.to_datetime(df[i],errors='coerce',format="%Y-%m-%d")

Let's check data before splitting

In [415]:
df.head()

,id,activity_new,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,48ada52261e7cf58715202705a0451c9,esoiiifxdlbkcsluxmfuacbdckommixw,lmkebamcaaclubfxadlmueccxoimlema,309275,0,10025,2012-11-07,2016-11-06,2012-11-07,2015-11-09,26520.30,10025,0.0,359.29,0.095919,0.088347,58.995952,f,831.80,-41.76,-41.76,1,1732.36,3,ldkssxwpmemidmecebumciepifcamkci,180.000,0
1,d29c2c54acc38ff3c0614d0a653813dd,NaN,NaN,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,NaN,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
3,bba03439a292a1e166f80264c16191cb,NaN,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,568bb38a1afd7c0fc49c77b3789b59a3,sfisfxfcocfpcmckuekokxuseixdaoeu,foosdfpfkusacimwkcsosbicdxkicaua,121335,0,12400,2010-04-08,2016-04-08,2010-04-08,2015-04-12,10865.02,12400,0.0,170.74,0.110083,0.093746,40.606701,f,1052.37,-3.18,-3.18,1,823.18,6,lxidpiddsbxsbosboudacockeimpuepw,75.000,0


In [416]:
df.shape

(15674, 27)

In [417]:
hist.head()

,id,price_date,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


# **1. Create new features**

In this step, we are going to create new features based on our data. Those new features will be a driver to churn prediction.



## New features with dates

In [418]:
#Proportion of years to end the contract

df['years_end']=(df['date_end']-df['date_renewal'])

df['years_end']=df.years_end.astype('str')

df.years_end=df.years_end.str.split(' ').apply(lambda x: x[0]).astype('int64')/365

#Proportion of years from activation to modification

df['years_mod']=(df['date_modif_prod']-df['date_activ'])

df['years_mod']=df.years_mod.astype('str')

df.years_mod=df.years_mod.str.split(' ').apply(lambda x: x[0]).astype('int64')/365

#Check if new features are Ok
print(df.years_end.isna().sum())
print(df.years_mod.isna().sum())

0
0


## New features with consumption and forecast

In [419]:
#Total consumption (Electricity and Gas)

df['total_cons']=(df['cons_12m']-df['cons_gas_12m'])



In [420]:
#Forecasted price energy and pow

group_hist=hist.groupby('id').mean()
group_hist=group_hist[['price_p3_var','price_p2_fix','price_p3_fix']]
group_hist.columns=['forecast_price_energy_p3','forecast_price_pow_p2','forecast_price_pow_p3']
group_hist

#Merge to customer data

df=df.merge(group_hist,on='id')

#Average forecasted energy and power price

df['avg_fct_ene']=(df['forecast_price_energy_p1']+df['forecast_price_energy_p2']+df['forecast_price_energy_p3'])/3
df['avg_fct_pow']=(df['forecast_price_pow_p1']+df['forecast_price_pow_p2']+df['forecast_price_pow_p3'])/3



## Drop features and Correlation

Based on the feature engineering process we just performed, we have to drop some features.

In addition, to improve the future performance of the model, we are  going to drop features whose correlation with themselves are high.



In [421]:

#Variables to delete

to_drop=['date_end','date_renewal','date_modif_prod','date_activ','cons_12m','cons_gas_12m','forecast_price_energy_p1','forecast_price_energy_p2','forecast_price_energy_p3',
         'forecast_price_pow_p1','forecast_price_pow_p2','forecast_price_pow_p3',
         'forecast_cons_12m','forecast_cons_year','margin_gross_pow_ele']

df.drop(columns=to_drop,index=1,inplace=True)


## Grouping Categories

Let's explore categorical data and discover how we can handle it.

In [422]:
#Extract categorical features

categorical=[feature for feature in df.columns if df[feature].dtype =='object' and feature!='id']
print(categorical)

for i in categorical:
  print(df[i].value_counts())
  print('===='*10)

['activity_new', 'channel_sales', 'has_gas', 'origin_up']
apdekpcbwosbxepsfxclislboipuxpop    1528
kkklcdamwfafdcfwofuscwfwadblfmce     420
kwuslieomapmswolewpobpplkaooaaew     226
fmwdwsxillemwbbwelxsampiuwwpcdcb     214
ckfxocssowaeipxueikxcmaxdmcduxsa     186
                                    ... 
fxocpcbfplipxiokscwiuexkceoucmko       1
kkkmlicifclosfkbxodcmsaweebkolde       1
mloxfblllfoxllsffauklsewwfcfdlls       1
aplsmkockmiifibukmmmomommebkdpfk       1
cwkwaxadbfukekuspislmbipbkxdudla       1
Name: activity_new, Length: 417, dtype: int64
foosdfpfkusacimwkcsosbicdxkicaua    7113
lmkebamcaaclubfxadlmueccxoimlema    2038
usilxuppasemubllopkaafesmlibmsdf    1412
ewpakwlliwisiwduibdlfmalxowmwpci     943
sddiedcslfslkckwlfkdpoeeailfpeds      10
epumfxlbckeskwekxbiuasklxalciiuu       4
Name: channel_sales, dtype: int64
f    12795
t     2878
Name: has_gas, dtype: int64
lxidpiddsbxsbosboudacockeimpuepw    7590
kamkkxfxxuwbdslkwifmmcsiusiuosws    4488
ldkssxwpmemidmecebumciepifcamkci 

In [423]:
df[categorical].isna().sum()

activity_new     9304
channel_sales    4153
has_gas             0
origin_up           0
dtype: int64

Based on the previuos results we can see that is not necessary to group categorical data. The action we are going to take with feature ```channel_sales``` is to replace missing values with category ```other```

In [424]:
#Replacing NaN in channel_sales with 'other'

df.channel_sales.fillna('other',inplace=True)


## Extracting dates

In the previous EDA we saw that dates, months and years don't show that churn has an specific trend, for that reason, we created features based on dates previously.

# **2. Split data into Train and Test (customer data)**

This is very important to do before any kind of actions with data liker transformations, outlier detection, etc.

Transformations, scaling, outlier detection must be fit or perform in train set and then in test set we should tranform test data based on the results of train data.

Before moving with Splitting we are going to delete the feature ```activity_new``` because it has almost 60% of missing data.

In [425]:
#Drop activity_new

df.drop(columns='activity_new',inplace=True)

## 2.1 Getting dummy features

In [426]:
X=pd.get_dummies(X,drop_first=True)

In [427]:
#Split data

features=[x for x in df.columns if x!='id' and x!='churn']
X=df.loc[:,features]


y=df[['churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# **3. Outlier detection**

Based on EDA, we saw that almost all the numeric features were quite skwed,so, we have have remove outlier to ensure the best performance while modeling.

In [428]:
#Extract numeric features

numeric=[feature for feature in X.columns if X[feature].dtype =='int64' or X[feature].dtype=='float64' ]
numeric


['cons_last_month',
 'forecast_discount_energy',
 'forecast_meter_rent_12m',
 'imp_cons',
 'margin_net_pow_ele',
 'nb_prod_act',
 'net_margin',
 'num_years_antig',
 'pow_max',
 'years_end',
 'years_mod',
 'total_cons',
 'avg_fct_ene',
 'avg_fct_pow']

We can conclude from the EDA that the variables related to consumption and forecast are highly skewed to the right, for that reason, we are going to calculate thresholds for every numeric feature with percentiles 0.001 and 0.999



In [429]:
#Extracting percentiles

lower=[]
for i in numeric:
  lower.append(X_train[i].quantile(0.01))

upper=[]

for i in numeric:
  upper.append(X_train[i].quantile(0.99))

print(lower)
print(upper)

#Save X shape
X_shape=X_train.shape
print(X_shape)  

[0.0, 0.0, 0.0, 0.0, -15.4578, 1.0, 0.0, 3.0, 10.307400000000001, 0.9616438356164384, 0.0, -240707.18, 0.04610166666666666, 14.77045932]
[469210.0, 30.0, 359.25, 1873.6435999999874, 104.8355999999999, 4.0, 1314.1483999999955, 11.0, 100.0, 1.8373424657534225, 9.594520547945205, 3185835.0, 0.12289311143939383, 34.608155466666666]
(12538, 17)


In [430]:
#Removing outliers


for feature,low,up in zip(numeric,lower,upper):
 #print(feature,low,up)
 X_train=X_train[(X_train[feature]<=up)&(X_train[feature]>=low)]
 #print(algo.shape)

#New Shape of X_train
print(X_train.shape)

index=X_train.index
print(index)

y_train=y_train.loc[index,:]
print(y_train.shape)

(10868, 17)
Int64Index([15616,  6073,  7721, 12617, 12413,   747, 11741,  2249,  7777,
             5496,
            ...
             9786, 13436, 14443,  7764, 15378, 14055,  5219,  1347, 11647,
             3583],
           dtype='int64', length=10868)
(10868, 1)


# **4. Possible transformations**

Our data has negative values and we want to have data more Gaussian-like.

In this case, we are going to use the power transformer with scikit learn.

In [431]:
#Initialize the transformer

transformer=PowerTransformer()

#Fit the transformer

transformer.fit(X_train[numeric])

#Transform Train and test data

train_numerical_transform=transformer.transform(X_train[numeric])
test_numerical_transform=transformer.transform(X_test[numeric])

#Replace transform data in original X's data

X_train[numeric]=train_numerical_transform
X_test[numeric]=test_numerical_transform

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


# **5. Scaling features**

In [432]:
scaler=StandardScaler()

#Fit the transformer

scaler.fit(X_train[numeric])

#Transform Train and test data

train_numerical_scaled=scaler.transform(X_train[numeric])
test_numerical_scaled=scaler.transform(X_test[numeric])

#Replace transform data in original X's data

X_train[numeric]=train_numerical_scaled
X_test[numeric]=test_numerical_scaled

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


#Export datasets

In [433]:
X_train.to_csv('/content/drive/MyDrive/Colab Notebooks/BCG/X_train.csv',index=False)
X_test.to_csv('/content/drive/MyDrive/Colab Notebooks/BCG/X_test.csv',index=False)
y_train.to_csv('/content/drive/MyDrive/Colab Notebooks/BCG/y_train.csv',index=False)
y_test.to_csv('/content/drive/MyDrive/Colab Notebooks/BCG/y_test.csv',index=False)